In [53]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [54]:
# load dataset (note that dataset is seperated by ';' rather than usual ',')
data = pd.read_csv('WindData.csv', sep = ';')
data


,Date,Wind Production PZ_1,Wind Production PZ_2,Station1_WND,Station1_DD,Station2_WND,Station2_DD,Station3_WND,Station3_DD,Station4_WND,...,Station8_WND,Station8_DD,Station9_WND,Station9_DD,Station10_WND,Station10_DD,Station11_WND,Station11_DD,Station12_WND,Station12_DD
0,01/01/2016 00:00,"1396,847852","122,7055319","9,6",191,"6,1",179,7,224,"8,4",...,"12,4",199,"7,7",228,"6,1",195,"5,7",184,"10,2",202
1,01/01/2016 01:00,"1304,469541","105,1476672","9,8",188,"6,1",177,"6,6",227,"8,3",...,"13,2",197,"7,3",231,6,193,"5,3",185,"10,7",200
2,01/01/2016 02:00,"1201,416846","80,806082","9,9",188,"6,6",174,"6,3",231,"8,3",...,"13,4",197,"6,9",235,"6,3",193,"5,3",188,"10,4",199
3,01/01/2016 03:00,"1198,324268","67,63768345",10,187,"7,1",172,6,234,"8,4",...,"13,6",197,"6,6",239,"6,7",193,"5,2",191,"10,1",197
4,01/01/2016 04:00,"1113,926804","51,57622764","9,8",188,"7,5",172,"6,1",232,"8,5",...,13,202,"6,4",237,7,193,"4,8",195,"9,7",197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,31/12/2016 19:00,"3049,62372","843,204789","12,7",250,"12,5",249,"13,1",244,"12,9",...,"14,5",255,"14,2",239,"12,1",251,"10,9",254,11,244
8780,31/12/2016 20:00,"3045,754898","848,66082","12,7",250,"12,2",248,13,244,"12,7",...,"14,8",253,"14,1",240,12,250,"10,9",253,"11,4",245
8781,31/12/2016 21:00,"3091,585559","845,4864019","12,8",249,12,247,13,244,"12,6",...,"15,2",252,"14,1",241,"11,9",248,11,252,"11,8",246
8782,31/12/2016 22:00,"3092,379163","821,1822639","12,7",249,"11,5",248,13,245,"12,4",...,"15,3",251,14,242,"11,8",247,11,252,12,246


0       1396847852
1       1304469541
2       1201416846
3       1198324268
4       1113926804
           ...    
8779     304962372
8780    3045754898
8781    3091585559
8782    3092379163
8783    2935443872
Name: Wind Production PZ_1, Length: 8784, dtype: object

### Key points about the dataset
- Actual production for two regions (MW)  =  (Wind Production PZ_1, Wind Production PZ_2) columns
- forecasted wind speed (m/s) = WND columns
- forecasted wind direction (degrees) = DD columns


In [55]:
# check data types are correct
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date                  8784 non-null   object
 1   Wind Production PZ_1  8784 non-null   object
 2   Wind Production PZ_2  8784 non-null   object
 3   Station1_WND          8784 non-null   object
 4   Station1_DD           8784 non-null   int64 
 5   Station2_WND          8784 non-null   object
 6   Station2_DD           8784 non-null   object
 7   Station3_WND          8784 non-null   object
 8   Station3_DD           8784 non-null   object
 9   Station4_WND          8784 non-null   object
 10  Station4_DD           8784 non-null   object
 11  Station5_WND          8784 non-null   object
 12  Station5_DD           8784 non-null   int64 
 13  Station6_WND          8784 non-null   object
 14  Station6_DD           8784 non-null   int64 
 15  Station7_WND          8784 non-null   

In [56]:
# convert columns to numeric data types
data['Date'] = pd.to_datetime(data['Date'], format="%d/%m/%Y %H:%M")

# commas causing a problem with Wind Production columns
data['Wind Production PZ_1'] = data['Wind Production PZ_1'].str.replace(',','')
data['Wind Production PZ_1'] = pd.to_numeric(data['Wind Production PZ_1'],errors='coerce')
data['Wind Production PZ_2'] = data['Wind Production PZ_2'].str.replace(',','')
data['Wind Production PZ_2'] = pd.to_numeric(data['Wind Production PZ_2'],errors='coerce')
for i in range(1,13):
    data[f'Station{i}_DD'] = pd.to_numeric(data[f'Station{i}_DD'],errors='coerce')
for i in range(1,13):
    data[f'Station{i}_WND'] = pd.to_numeric(data[f'Station{i}_DD'],errors='coerce')


In [62]:
# check for null values
data.isnull().sum()

# number of null values low so drop rows containing null values
data = data.dropna()


Date                    0
Wind Production PZ_1    0
Wind Production PZ_2    0
Station1_WND            0
Station1_DD             0
Station2_WND            0
Station2_DD             0
Station3_WND            0
Station3_DD             0
Station4_WND            0
Station4_DD             0
Station5_WND            0
Station5_DD             0
Station6_WND            0
Station6_DD             0
Station7_WND            0
Station7_DD             0
Station8_WND            0
Station8_DD             0
Station9_WND            0
Station9_DD             0
Station10_WND           0
Station10_DD            0
Station11_WND           0
Station11_DD            0
Station12_WND           0
Station12_DD            0
dtype: int64

Now data has been cleaned it is time to explore/visualise data